### Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install srai[all]
!pip install contextily
!pip install alphashape

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.2/118.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/543.2 kB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.3/812.3 kB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
import zipfile
import srai
import os
from PIL import Image
import glob
import contextily as ctx
import alphashape

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


#To create embeddings
from srai.loaders import OSMOnlineLoader, OSMWayLoader, OSMPbfLoader
from srai.regionalizers import geocode_to_region_gdf, S2Regionalizer
from srai.plotting import plot_regions, plot_numeric_data
from srai.embedders import CountEmbedder, ContextualCountEmbedder,Hex2VecEmbedder, Highway2VecEmbedder
from srai.joiners import IntersectionJoiner
from srai.loaders.osm_loaders.filters import HEX2VEC_FILTER
from srai.neighbourhoods.h3_neighbourhood import H3Neighbourhood
from srai.regionalizers import H3Regionalizer, geocode_to_region_gdf

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Hex2vec Embedder

CREATE STUDY AREA

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/UC-TESIS/data/vars_perceptuales_santiago.csv')
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
gdf = gdf.set_crs(epsg=4326)
gdf_rm = gdf.copy().drop(columns=['lat', 'lon', 'latlong'])
gdf_rm['feature_id'] = range(1, len(gdf) + 1)
cols = gdf_rm.columns.tolist()
# Move 'unique_id' to the beginning
cols.insert(0, cols.pop(cols.index('feature_id')))
# Reorder the DataFrame columns
gdf_rm = gdf_rm[cols]
# Extraer los puntos de la columna 'geometry'
points = list(gdf_rm.geometry)
points = np.array([[point.x, point.y] for point in gdf_rm.geometry])
# Calcular el alpha shape
alpha = 100
alpha_shape = alphashape.alphashape(points, alpha)

# Crear un nuevo GeoDataFrame con el alpha shape
gdf_alpha_shape = gpd.GeoDataFrame(geometry=[alpha_shape])

In [ ]:
gdf_alpha_shape['region_id'] = "Santiago Metropolitan Region, Chile"
gdf_alpha_shape = gdf_alpha_shape.set_crs(epsg=4326)
study_area2 = gdf_alpha_shape
study_area2

,geometry,region_id
0,"MULTIPOLYGON (((-70.76705 -33.67659, -70.76581...","Santiago Metropolitan Region, Chile"


EXTRACT OSM FEATURES

In [ ]:
loader = OSMPbfLoader()
scl_features_gdf = loader.load(study_area2, HEX2VEC_FILTER)
scl_features_gdf

/usr/local/lib/python3.10/dist-packages/srai/loaders/osm_loaders/osm_pbf_loader.py:128: FutureWarning: Use `convert_geometry_to_geodataframe` instead. Deprecated since 0.8.1 version.
  features_gdf = pbf_reader.get_features_gdf_from_geometry(
100%|████████████████████████████████████████| 304M/304M [00:00<00:00, 217GB/s]
SHA256 hash of downloaded file: 6213ab68931a1d6d7831a83f9cf9671f4f9d07a8b8445c8106b041c1e08b86d2
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


Output()

Finished operation in 0:01:32

,geometry,aeroway,amenity,building,healthcare,historic,landuse,leisure,military,natural,office,shop,sport,tourism,water,waterway
feature_id,,,,,,,,,,,,,,,,
node/298973459,POINT (-70.60496 -33.41657),None,None,None,None,None,None,None,None,None,None,None,None,hotel,None,None
node/301003861,POINT (-70.60580 -33.51045),None,None,None,None,None,retail,None,None,None,None,doityourself,None,None,None,None
node/1286583425,POINT (-70.51461 -33.38582),None,None,None,None,None,None,None,None,peak,None,None,None,None,None,None
node/1286674662,POINT (-70.53765 -33.42938),None,parking,None,None,None,None,None,None,None,None,None,None,None,None,None
node/1286674768,POINT (-70.53762 -33.42807),None,parking,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
way/1125064988,"POLYGON ((-70.74306 -33.45711, -70.74296 -33.4...",None,None,hospital,None,None,None,None,None,None,None,None,None,None,None,None
way/1125064989,"POLYGON ((-70.74295 -33.45658, -70.74290 -33.4...",None,clinic,None,None,None,None,None,None,None,None,None,None,None,None,None
way/1125064993,"POLYGON ((-70.74283 -33.45669, -70.74254 -33.4...",None,parking,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
clipped_features_gdf = scl_features_gdf.clip(study_area2.geometry.unary_union)

VISUALIZE FEATURES

In [ ]:
ax = study_area2.plot(color="lavender", figsize=(16, 16))

# plot water
clipped_features_gdf.dropna(subset=["water", "waterway"], how="all").plot(
    ax=ax, color="deepskyblue"
)

# plot greenery
clipped_features_gdf[
    clipped_features_gdf["landuse"].isin(
        ["grass", "orchard", "flowerbed", "forest", "greenfield", "meadow"]
    )
].plot(ax=ax, color="mediumseagreen")

# plot amenities
clipped_features_gdf[
    clipped_features_gdf["amenity"].isin(
        ["restaurant", "veterinary", "food_court", "cafe", "hospital", "university",
        "police","childcare","social_centre"]
    )
].plot(ax=ax, color="orange")

# plot buildings
clipped_features_gdf.dropna(subset=["building"], how="all").plot(
    ax=ax, color="dimgray", markersize=0.1
)

xmin, ymin, xmax, ymax = study_area2.total_bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

ax.set_axis_off()

PREPARE INPUTS FOR EMBEDDER

REGIONALIZER

In [ ]:
regionalizer = H3Regionalizer(resolution=10, buffer=True)
regions_gdf_rm_10 = regionalizer.transform(study_area2)

In [ ]:
regions_gdf_rm_10

,geometry
region_id,
8ab2c5435807fff,"POLYGON ((-70.84879 -33.44433, -70.84943 -33.4..."
8ab2c5184367fff,"POLYGON ((-70.50623 -33.33952, -70.50686 -33.3..."
8ab2c54614b7fff,"POLYGON ((-70.65584 -33.52980, -70.65647 -33.5..."
8ab2c5409c37fff,"POLYGON ((-70.78141 -33.54326, -70.78204 -33.5..."
8ab2c5568db7fff,"POLYGON ((-70.61965 -33.40993, -70.62028 -33.4..."
...,...
8ab2c5cde6d7fff,"POLYGON ((-70.71590 -33.34005, -70.71654 -33.3..."
8ab2c508540ffff,"POLYGON ((-70.51664 -33.48781, -70.51727 -33.4..."
8ab2c550d39ffff,"POLYGON ((-70.75756 -33.39558, -70.75819 -33.3..."


Filter regions

In [ ]:
filtered_gdf_R10_N3 = pd.read_csv('/content/drive/MyDrive/UC-TESIS/data/filtered_gdf_R10_N3.csv')
filtered_gdf_R10_N3.drop(columns=['feature_id','geometry'], inplace=True)
filtered_gdf_R10_N3

,region_id
0,8ab2c51a274ffff
1,8ab2c51a274ffff
2,8ab2c51a274ffff
3,8ab2c51a274ffff
4,8ab2c51a274ffff
...,...
83723,8ab2c54dd80ffff
83724,8ab2c54dd80ffff
83725,8ab2c57a268ffff
83726,8ab2c57a268ffff


In [ ]:
regions_gdf_rm_10_no_index= regions_gdf_rm_10.reset_index()
regions_gdf_rm_10_no_index

,region_id,geometry
0,8ab2c5435807fff,"POLYGON ((-70.84879 -33.44433, -70.84943 -33.4..."
1,8ab2c5184367fff,"POLYGON ((-70.50623 -33.33952, -70.50686 -33.3..."
2,8ab2c54614b7fff,"POLYGON ((-70.65584 -33.52980, -70.65647 -33.5..."
3,8ab2c5409c37fff,"POLYGON ((-70.78141 -33.54326, -70.78204 -33.5..."
4,8ab2c5568db7fff,"POLYGON ((-70.61965 -33.40993, -70.62028 -33.4..."
...,...,...
87105,8ab2c5cde6d7fff,"POLYGON ((-70.71590 -33.34005, -70.71654 -33.3..."
87106,8ab2c508540ffff,"POLYGON ((-70.51664 -33.48781, -70.51727 -33.4..."
87107,8ab2c550d39ffff,"POLYGON ((-70.75756 -33.39558, -70.75819 -33.3..."
87108,8ab2c557090ffff,"POLYGON ((-70.71131 -33.38041, -70.71194 -33.3..."


In [ ]:
regions_gdf_rm_10_filtered = regions_gdf_rm_10_no_index[regions_gdf_rm_10_no_index['region_id'].isin(filtered_gdf_R10_N3['region_id'])]
regions_gdf_rm_10_filtered = regions_gdf_rm_10_filtered.set_index('region_id')
regions_gdf_rm_10_filtered

,geometry
region_id,
8ab2c54614b7fff,"POLYGON ((-70.65584 -33.52980, -70.65647 -33.5..."
8ab2c5409c37fff,"POLYGON ((-70.78141 -33.54326, -70.78204 -33.5..."
8ab2c5735c27fff,"POLYGON ((-70.56603 -33.57904, -70.56667 -33.5..."
8ab2c5470d37fff,"POLYGON ((-70.73566 -33.51980, -70.73629 -33.5..."
8ab2c51982d7fff,"POLYGON ((-70.53993 -33.41523, -70.54056 -33.4..."
...,...
8ab2c5440607fff,"POLYGON ((-70.69344 -33.57521, -70.69407 -33.5..."
8ab2c5735ba7fff,"POLYGON ((-70.55387 -33.58165, -70.55450 -33.5..."
8ab2c519132ffff,"POLYGON ((-70.55562 -33.37959, -70.55625 -33.3..."


24220 regions with 3 or more photopoints

CREATE EMBEDDINGS

In [ ]:
#regionalizer = H3Regionalizer(resolution=10, buffer=True)
joiner = IntersectionJoiner()
area = study_area2

features = loader.load(area, HEX2VEC_FILTER)
regions = regions_gdf_rm_10_filtered
joint = joiner.transform(regions, features)

embedder = Hex2VecEmbedder([150, 75, 25])
neighbourhood = H3Neighbourhood(regions_gdf_rm_10_filtered)

#fit_transform
h3_embeddings_h2vec = embedder.fit_transform(
          regions,
          features,
          joint,
          neighbourhood,
          #trainer_kwargs={"max_epochs": 10, "accelerator": "gpu"},
          batch_size=128)

folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
plot_numeric_data(regions, 0, h3_embeddings_h2vec, map=folium_map)

Output hidden; open in https://colab.research.google.com to view.

Loop trying different embedder sizes

In [ ]:
joiner = IntersectionJoiner()
area = study_area2

features = loader.load(area, HEX2VEC_FILTER)
regions = regions_gdf_rm_10_filtered
joint = joiner.transform(regions, features)
neighbourhood = H3Neighbourhood(regions_gdf_rm_10_filtered)

embedder_sizes = [[150, 75, 100], [150, 75, 125], [150, 75, 150]]  # Define different embedder sizes

for sizes in embedder_sizes:
    embedder = Hex2VecEmbedder(sizes)
    embedding_name = "h3_embeddings_" + "_".join(map(str, sizes))  # Create a name based on sizes

    # Fit and transform using the current embedder
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        embeddings = embedder.fit_transform(
            regions,
            features,
            joint,
            neighbourhood,
            trainer_kwargs={"max_epochs":50, "accelerator": "gpu"},
            batch_size=128)

    # Store or use the embeddings with the corresponding name
    # Example:
    globals()[embedding_name] = embeddings
    print(f"Created embeddings with name: {embedding_name}")

/usr/local/lib/python3.10/dist-packages/srai/loaders/osm_loaders/osm_pbf_loader.py:128: FutureWarning: Use `convert_geometry_to_geodataframe` instead. Deprecated since 0.8.1 version.
  features_gdf = pbf_reader.get_features_gdf_from_geometry(
100%|██████████| 24220/24220 [00:00<00:00, 29636.51it/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 96.2 K | train
-----------------------------------------------
96.2 K    Trainable params
0         Non-trainable params
96.2 K    Total params
0.385     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


Created embeddings with name: h3_embeddings_150_75_100


100%|██████████| 24220/24220 [00:00<00:00, 30050.72it/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 98.1 K | train
-----------------------------------------------
98.1 K    Trainable params
0         Non-trainable params
98.1 K    Total params
0.392     Total estimated model params size (MB)


...

Created embeddings with name: h3_embeddings_150_75_125


100%|██████████| 24220/24220 [00:00<00:00, 30081.77it/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params  | Mode 
------------------------------------------------
0 | encoder | Sequential | 100.0 K | train
------------------------------------------------
100.0 K   Trainable params
0         Non-trainable params
100.0 K   Total params
0.400     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


Created embeddings with name: h3_embeddings_150_75_150


In [ ]:
h3_embeddings_150_75_150.to_csv('/content/drive/MyDrive/UC-TESIS/data/embeddings/h3_embeddings_150_75_150.csv')

h3_embeddings_150_75_25

In [ ]:
h3_embeddings_150_75_25

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
region_id,,,,,,,,,,,,,,,,,,,,,
8ab2c54614b7fff,-0.071019,0.035029,0.203784,-0.024328,-0.497790,0.223362,0.400645,-0.107379,0.068013,-0.379598,...,0.327683,-0.070662,0.231740,0.072074,0.017460,-0.483366,0.106648,-0.184167,0.112329,-0.049347
8ab2c5409c37fff,0.268548,0.334361,0.157701,-0.222651,0.231190,0.394530,-0.975918,-0.096894,0.195916,0.091382,...,0.026791,0.099589,-0.157671,-0.725638,0.213838,-0.552927,-0.531080,-0.659286,-0.216646,-0.945521
8ab2c5735c27fff,0.619432,-0.528625,-0.848479,0.712676,-0.637829,-0.016624,-0.385903,1.430742,-0.695935,0.884306,...,-0.274605,-0.696368,-0.483474,-0.175802,-0.032254,-0.221234,-0.014829,-0.755150,0.637143,0.258227
8ab2c5470d37fff,-0.118786,-0.139472,0.185815,-0.428418,0.473278,0.371524,0.156727,0.321230,-0.583839,-0.031883,...,-1.213616,0.335448,0.411573,0.060988,-0.026058,0.480535,-0.702561,0.165846,0.295005,0.156483
8ab2c51982d7fff,-0.047976,-0.010160,0.273949,-1.022394,0.643530,0.659075,-1.406568,-0.518560,0.026370,-0.345660,...,-0.424221,-0.854667,-0.888435,0.080287,0.701661,0.197022,0.074844,-0.280483,-0.511988,-0.110741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8ab2c5440607fff,0.520605,0.360582,0.491219,0.035904,-0.259304,-0.334493,-0.002457,0.012579,0.158216,-0.179374,...,-1.257912,-0.484045,-0.013793,-0.177689,0.404953,0.297659,-0.144526,0.089387,-0.359202,0.449188
8ab2c5735ba7fff,-0.071019,0.035029,0.203784,-0.024328,-0.497790,0.223362,0.400645,-0.107379,0.068013,-0.379598,...,0.327683,-0.070662,0.231740,0.072074,0.017460,-0.483366,0.106648,-0.184167,0.112329,-0.049347
8ab2c519132ffff,-0.071019,0.035029,0.203784,-0.024328,-0.497790,0.223362,0.400645,-0.107379,0.068013,-0.379598,...,0.327683,-0.070662,0.231740,0.072074,0.017460,-0.483366,0.106648,-0.184167,0.112329,-0.049347


In [ ]:
folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
plot_numeric_data(regions, 0, h3_embeddings_150_75_25, map=folium_map)

Output hidden; open in https://colab.research.google.com to view.

h3_embeddings_150_75_50

In [ ]:
h3_embeddings_150_75_50

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
region_id,,,,,,,,,,,,,,,,,,,,,
8ab2c54614b7fff,-0.114313,0.020671,-0.040652,0.015408,-0.262271,-0.166279,-0.429452,-0.022514,0.350163,-0.104940,...,0.156261,-0.132325,-0.128945,0.156389,0.048006,-0.187276,0.055415,-0.154178,0.060354,0.122388
8ab2c5409c37fff,-0.576374,-0.255128,-0.934314,-0.164343,-0.123671,0.058546,-0.069132,0.449853,0.308786,0.196857,...,-0.131234,-0.937947,0.111949,-0.245396,0.674050,0.590559,-0.098312,0.032940,0.470065,-0.098238
8ab2c5735c27fff,0.201672,0.430278,-0.077801,0.302460,-0.061443,-0.207122,0.154171,-0.111821,-0.317251,0.145342,...,0.568890,-0.078769,0.451521,0.010797,-0.144747,-0.225121,0.469349,1.097817,-1.047041,-0.019882
8ab2c5470d37fff,0.207020,-0.106814,-0.001290,0.215326,-0.471588,0.293057,0.408622,-0.182512,-0.197646,-0.099400,...,-0.327405,0.322140,0.048162,-0.018927,-0.382849,0.106345,0.172986,0.147097,-0.260188,0.663659
8ab2c51982d7fff,-0.628367,0.095917,-0.372501,-0.142626,-0.227971,-0.130558,0.360125,0.266565,-0.439376,0.831126,...,0.307525,-0.093052,0.580987,0.084759,-0.491205,1.639907,-0.193301,0.315952,0.458435,-0.264330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8ab2c5440607fff,0.371540,-0.233017,0.187141,-0.297901,-0.122089,0.549419,0.021063,-0.058300,-0.528785,-0.424006,...,-0.137458,0.851793,0.317988,-0.041839,0.297002,0.378061,-0.286429,-0.133449,-0.477818,0.217868
8ab2c5735ba7fff,-0.114313,0.020671,-0.040652,0.015408,-0.262271,-0.166279,-0.429452,-0.022514,0.350163,-0.104940,...,0.156261,-0.132325,-0.128945,0.156389,0.048006,-0.187276,0.055415,-0.154178,0.060354,0.122388
8ab2c519132ffff,-0.114313,0.020671,-0.040652,0.015408,-0.262271,-0.166279,-0.429452,-0.022514,0.350163,-0.104940,...,0.156261,-0.132325,-0.128945,0.156389,0.048006,-0.187276,0.055415,-0.154178,0.060354,0.122388


In [ ]:
folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
plot_numeric_data(regions, 0, h3_embeddings_150_75_50, map=folium_map)

Output hidden; open in https://colab.research.google.com to view.

h3_embeddings_150_75_75

In [ ]:
h3_embeddings_150_75_75

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
region_id,,,,,,,,,,,,,,,,,,,,,
8ab2c54614b7fff,0.017901,0.056993,0.035953,0.040018,-0.055870,-0.142250,-0.043834,-0.070997,-0.026111,-0.006281,...,-0.018467,-0.004865,0.237714,0.326294,0.020518,-0.062686,-0.042715,0.099404,-0.014217,-0.139572
8ab2c5409c37fff,-0.145355,0.178755,0.004304,-0.128986,0.156285,-0.079129,-0.163455,-0.285920,-0.169034,-0.294594,...,-0.243557,0.000632,-0.023831,0.127698,-0.455067,0.148532,-0.049172,-0.000904,0.109383,-0.165811
8ab2c5735c27fff,-1.013563,-0.100562,-0.377272,0.562718,0.157968,0.831198,-0.506169,0.258162,-0.051279,0.260607,...,0.215660,0.721094,0.348316,-0.411984,-0.749191,0.723928,0.400240,-0.235033,-0.563700,0.726097
8ab2c5470d37fff,0.272572,-0.211327,0.140348,0.133513,-0.069328,0.227476,-0.100565,-0.038315,0.115610,-0.286060,...,0.044032,0.484444,-0.592968,-0.301831,-0.326002,-0.106915,-0.219240,0.381943,-0.055726,-0.303097
8ab2c51982d7fff,-0.683943,-0.196676,-0.125761,-0.193637,0.098454,-0.157441,0.082356,0.023404,-0.088587,0.251796,...,0.097162,-0.046992,0.573615,0.071486,0.481251,-0.619875,0.363252,-0.511372,-0.056549,0.381586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8ab2c5440607fff,0.450711,-0.241096,0.247476,-0.184891,-0.043256,0.137342,-0.393495,0.340668,0.277873,0.313847,...,0.009105,-0.093170,0.069810,-0.279551,-0.221418,-0.176748,-0.508331,0.109644,-0.282956,0.438303
8ab2c5735ba7fff,0.017901,0.056993,0.035953,0.040018,-0.055870,-0.142250,-0.043834,-0.070997,-0.026111,-0.006281,...,-0.018467,-0.004865,0.237714,0.326294,0.020518,-0.062686,-0.042715,0.099404,-0.014217,-0.139572
8ab2c519132ffff,0.017901,0.056993,0.035953,0.040018,-0.055870,-0.142250,-0.043834,-0.070997,-0.026111,-0.006281,...,-0.018467,-0.004865,0.237714,0.326294,0.020518,-0.062686,-0.042715,0.099404,-0.014217,-0.139572


In [ ]:
folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
plot_numeric_data(regions, 0, h3_embeddings_150_75_75, map=folium_map)

Output hidden; open in https://colab.research.google.com to view.

h3_embeddings_150_75_100

In [ ]:
folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
plot_numeric_data(regions, 0, h3_embeddings_150_75_100, map=folium_map)


Output hidden; open in https://colab.research.google.com to view.

h3_embeddings_150_75_125

In [ ]:
folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
plot_numeric_data(regions, 0, h3_embeddings_150_75_125, map=folium_map)

Output hidden; open in https://colab.research.google.com to view.

h3_embeddings_150_75_150

In [ ]:
folium_map = plot_regions(area, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
plot_numeric_data(regions, 0, h3_embeddings_150_75_150, map=folium_map)

Output hidden; open in https://colab.research.google.com to view.



---

---





---



# New Section